# Define Tools

We first need to create the tools we want to use. Our main tool of choice will be Tavily - a search engine. We have a built-in tool in LangChain to easily use Tavily search engine as tool.

In [1]:
from dotenv import load_dotenv
load_dotenv()

from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'title': 'Tuesday, May 20, 2025. San Francisco, CA - Weather Forecast', 'url': 'https://weathershogun.com/weather/usa/ca/san-francisco/480/may/2025-05-20', 'content': 'San Francisco, California Weather: Tuesday, May 20, 2025. Cloudy weather, overcast skies with clouds. Day 66°. Night 52°. Precipitation 0 %.', 'score': 0.94339466}, {'title': 'Weather in San Francisco in May 2025 (California)', 'url': 'https://world-weather.info/forecast/usa/san_francisco/may-2025/', 'content': 'Detailed ⚡ San Francisco Weather Forecast for May 2025 – day/night 🌡️ temperatures, precipitations – World-Weather ... Tuesday, 20 May. +52°. Day. +61', 'score': 0.91306627}]


# Usage Language Model (Google Gemini)

Next, let's learn how to use a language model to call tools. LangChain supports many different language models that you can use interchangably - select the one you want to use below!

In [5]:


from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")


from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi I am arafat")])
response.content

"Hi Arafat! It's nice to meet you. How can I help you today?"

# Model With Tool

In [6]:
model_with_tools = model.bind_tools(tools)

In [7]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hi there! How can I help you today?
ToolCalls: []


In [8]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'e7fa4682-1c59-4115-93b3-155f8ed6a0f6', 'type': 'tool_call'}]


# Create the agent

Now that we have defined the tools and the LLM, we can create the agent. We will be using LangGraph to construct the agent. Currently, we are using a high level interface to construct the agent, but the nice thing about LangGraph is that this high-level interface is backed by a low-level, highly controllable API in case you want to modify the agent logic.

In [9]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

# Run the Agent

We can now run the agent with a few queries! Note that for now, these are all stateless queries (it won't remember previous interactions). Note that the agent will return the final state at the end of the interaction (which includes any inputs, we will see later on how to get only the outputs).

In [10]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='f11a1c54-0519-42c0-8223-f37f211520c6'),
 AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--3025a596-5023-4112-bfb1-960f81e0e59d-0', usage_metadata={'input_tokens': 51, 'output_tokens': 11, 'total_tokens': 62, 'input_token_details': {'cache_read': 0}})]

In [11]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', additional_kwargs={}, response_metadata={}, id='87631447-cc93-4d60-a331-fcb68cdb3911'),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query": "weather in San Francisco"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--10adb3c1-ce1e-4b28-8b9b-e130a0f6644f-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': '28951b0f-0248-4974-bf1f-1deb384834e7', 'type': 'tool_call'}], usage_metadata={'input_tokens': 55, 'output_tokens': 13, 'total_tokens': 68, 'input_token_details': {'cache_read': 0}}),
 ToolMessage(content='[{"title": "Tuesday, May 20, 2025. San Francisco, CA - Weather Forecast", "url": "https://weathershogun.com/weather/usa/ca/san-francisco/480/may/2025-05-20", "content": "San Francisco, 

# Adding in memory

As mentioned earlier, this agent is stateless. This means it does not remember previous interactions. To give it memory we need to pass in a checkpointer. When passing in a checkpointer, we also have to pass in a thread_id when invoking the agent (so it knows which thread/conversation to resume from).

In [12]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [13]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [14]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hi Bob! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--a9df61df-954f-4245-ad5c-1f2114921a10-0', usage_metadata={'input_tokens': 53, 'output_tokens': 11, 'total_tokens': 64, 'input_token_details': {'cache_read': 0}})]}}
----


In [15]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--69e7d988-557e-479b-a9b8-32352847a916-0', usage_metadata={'input_tokens': 67, 'output_tokens': 6, 'total_tokens': 73, 'input_token_details': {'cache_read': 0}})]}}
----
